# Основная информация проекта

## Описание датасета

1. **Restaurant_id** — идентификационный номер ресторана / сети ресторанов;
2. **City** — город, в котором находится ресторан;
3. **Cuisine Style** — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
4. **Ranking** — место, которое занимает данный ресторан среди всех ресторанов своего города;
5. **Rating** — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
6. **Price Range** — диапазон цен в ресторане;
7. **Number of Reviews** — количество отзывов о ресторане;
8. **Reviews** — данные о двух отзывах, которые отображаются на сайте ресторана;
9. **URL_TA** — URL страницы ресторана на TripAdvisor;
10. **ID_TA** — идентификатор ресторана в базе данных TripAdvisor.

**Целевая переменная** - "Rating"

# Импорт библиотек и загрузка данных

In [534]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
import re

In [1072]:
df = pd.read_csv('main_task_new.csv')

# Функции для обработки и анализа данных

In [1066]:
def find_null(col):
    '''Функция подсчёта количества пустых значений в столбцах датасета'''
    empty_values = df[col].isna().sum()
    print(
        f"В столбце '{col}' количество пропусков = {empty_values}.")
    return empty_values

In [1067]:
def last_date(x):
    '''Функция для определния последнего дня отзыва'''
    if len(x) > 0:
        return x[0]
    else:
        return 0

In [1068]:
def row_to_datetime(row):
    '''Функция для преобразования данных в списке в формат даты'''
    new_row = []
    if len(row) > 0:
        for i in row:
            new_row.append(pd.to_datetime(i, errors='coerce'))
        return new_row
    else:
        return []

In [1069]:
def delta_datetime(row):
    '''Функция подсчёта разницы между датами'''
    if len(row) == 2:
        return (row[0]-row[1]).days
    else:
        return 0

In [1079]:
def parsing_all_columns_data(data):
    '''Функция для анализа всех столбцов на количество 
    уникальных значений и количество пропусков'''

    for col in data.columns:
        unique_values = len(data[col].value_counts())
        empty_values = data[col].isna().sum()
        print(
            f"В столбце '{col}' уникальных значений = {unique_values} ,а количество пропусков = {empty_values}.")
        print(
            f'Процент пропусков = {round(empty_values*100/df.shape[0],1)}%')

# Обзор данных

In [1075]:
display(df.head(5))
df.info()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [1076]:
# Вывод количества строк и столбцов таблицы
print(
    f'Датасет состоит из {df.shape[0]} строк и {df.shape[1]} столбцов')

Датасет состоит из 40000 строк и 10 столбцов


In [1080]:
parsing_all_columns_data(df)

В столбце 'Restaurant_id' уникальных значений = 11909 ,а количество пропусков = 0.
Процент пропусков = 0.0%
В столбце 'City' уникальных значений = 31 ,а количество пропусков = 0.
Процент пропусков = 0.0%
В столбце 'Cuisine Style' уникальных значений = 9007 ,а количество пропусков = 9283.
Процент пропусков = 23.2%
В столбце 'Ranking' уникальных значений = 11936 ,а количество пропусков = 0.
Процент пропусков = 0.0%
В столбце 'Rating' уникальных значений = 9 ,а количество пропусков = 0.
Процент пропусков = 0.0%
В столбце 'Price Range' уникальных значений = 3 ,а количество пропусков = 13886.
Процент пропусков = 34.7%
В столбце 'Number of Reviews' уникальных значений = 1459 ,а количество пропусков = 2543.
Процент пропусков = 6.4%
В столбце 'Reviews' уникальных значений = 33516 ,а количество пропусков = 0.
Процент пропусков = 0.0%
В столбце 'URL_TA' уникальных значений = 39980 ,а количество пропусков = 0.
Процент пропусков = 0.0%
В столбце 'ID_TA' уникальных значений = 39980 ,а количество пр

In [1081]:
# Приведём наименования колонок в общий вид
df.columns = map(str.lower, df.columns.str.replace(' ','_',regex=True))

# Работа с данными

# Вопросы

In [41]:
data['Price Range'].value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [62]:
len(data['City'].value_counts())

31

In [65]:
len(data['Cuisine Style'][0])

39

# 4.3 Вопросы о кухнях

In [1062]:
test = data.copy()

In [1064]:
test.columns = map(str.lower, test.columns.str.replace(' ','_',regex=True))

In [947]:
test.columns = test.columns.str.replace(' ','_',regex=True)

In [948]:
#data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x : re.findall(r'\w+',x) if not pd.isna(x) else ['Unknown'])

In [949]:
test['cuisine_style'] = test['cuisine_style'].apply(lambda x : re.sub(r"[]['']",'',x) if not pd.isna(x) else 'Unknown')

In [1065]:
test.head(5)

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


## Сколько типов кухонь представлено в наборе данных?

In [951]:
t = Counter(test['cuisine_style'].str.split(', ').sum())

In [952]:
len(t) - 1

125

## Какая кухня представлена в наибольшем количестве ресторанов?

In [953]:
t.most_common(1)

[('Vegetarian Friendly', 11189)]

In [954]:
# data.head(8)

## Какое среднее количество кухонь предлагается в одном ресторане?

In [955]:
test['cuisine_style'] = test['cuisine_style'].str.split(', ')

In [956]:
test.head(2)

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta
0,id_5569,Paris,"[European, French, International]",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,[Unknown],1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032


In [957]:
summ = 0
for i in test['cuisine_style']:
    summ += len(i)

In [958]:
summ/test.shape[0]

2.6224

# 4.4 Вопросы об отзывах

In [959]:
test['reviews'][0]

"[['Good food at your doorstep', 'A good hotel restaurant'], ['12/31/2017', '11/20/2017']]"

In [400]:
#test['Reviews'] = test['Reviews'].apply(lambda x : re.sub(r"[]['']",'',x) if not pd.isna(x) else 'Unknown')

In [530]:
# test['Reviews'].apply(lambda x : re.sub(r"['][']",'',x))

In [486]:
#pattern = re.compile('\d+\/\d+\/\d+')
#test['Reviews_date'] = test['Reviews'].apply(lambda x: pattern.findall(x))

In [471]:
#d = test['Reviews'].str.replace('[]['']','').str.split(', ')

In [487]:
#d[0][:2:]

In [678]:
#test['reviews'] = test['reviews'].apply(lambda x : re.sub(r"[]['']",'',x)).str.split(', ')

In [679]:
#print(dd[39995][:2:])
#print(dd[39995][2::])

['The real Italian experience!', 'Wonderful food']
['wonderful experience', '12/16/2017', '11/12/2017']


In [680]:
def test2(row):    
    return row[-int(len(row)/2)::]

In [681]:
def test3(row):
    return row[:int(len(row)/2):]

In [553]:
#test['reviews'].apply(test2)

0                              [12/31/2017, 11/20/2017]
1                              [07/06/2017, 06/19/2016]
2                              [01/08/2018, 01/06/2018]
3                                                    []
4                              [11/18/2017, 02/19/2017]
                              ...                      
39995    [wonderful experience, 12/16/2017, 11/12/2017]
39996                          [12/21/2017, 12/12/2017]
39997                          [11/03/2016, 04/12/2008]
39998                          [07/11/2017, 06/18/2017]
39999                          [01/31/2016, 07/04/2012]
Name: Reviews, Length: 40000, dtype: object

In [560]:
#test['reviews'].apply(test3)

0        [Good food at your doorstep, A good hotel rest...
1                [Unique cuisine, Delicious Nepalese food]
2                 [Catch up with friends, Not exceptional]
3                                                       []
4        [Best place to try a Bavarian food, Nice build...
                               ...                        
39995       [The real Italian experience!, Wonderful food]
39996    [Parisian atmosphere, Bit pricey but interesti...
39997    [Good by swedish standards, A hidden jewel in ...
39998    [Underground restaurant, Oldest Restaurant in ...
39999                          [Average, Nice an informal]
Name: Reviews, Length: 40000, dtype: object

In [960]:
pattern = re.compile('\d+\/\d+\/\d+')
test['reviews_date'] = test['reviews'].apply(lambda x: pattern.findall(x))

In [961]:
#pd.to_datetime(test['reviews_date']).dt.date

In [962]:
def last_date(x):
    if len(x) > 0:
        return x[0]
    else:
        return 0

In [963]:
test['last_reviews_date'] = test['reviews_date'].apply(last_date)

In [964]:
test['last_reviews_date'] = pd.to_datetime(test['last_reviews_date'],errors='coerce')

In [965]:
test['last_reviews_date'] = test['last_reviews_date'].apply(lambda x: datetime.strftime(x,'%Y-%m-%d'))

In [966]:
test.last_reviews_date.sort_values(ascending=False)

5635     2018-02-26
29251    2018-02-26
8135     2018-02-26
25943    2018-02-26
30334    2018-02-26
            ...    
21482    1970-01-01
5671     1970-01-01
21481    1970-01-01
21480    1970-01-01
4705     1970-01-01
Name: last_reviews_date, Length: 40000, dtype: object

In [832]:
'''
def dates_dif(row):
    if row is None:
        return None
    elif len(row) == 2:
        first = row[0]
        second = row[1]
        if first > second:
            dif = first - second
        else:
            dif = second - first
        dif = str(dif)
        dif = dif[:dif.find('day')]
        dif = int(dif)
        return dif
    else:
        return None
test['Difference between Dates'] = test['reviews_date'].apply(dates_dif)
'''

"\ndef dates_dif(row):\n    if row is None:\n        return None\n    elif len(row) == 2:\n        first = row[0]\n        second = row[1]\n        if first > second:\n            dif = first - second\n        else:\n            dif = second - first\n        dif = str(dif)\n        dif = dif[:dif.find('day')]\n        dif = int(dif)\n        return dif\n    else:\n        return None\ntest['Difference between Dates'] = test['reviews_date'].apply(dates_dif)\n"

In [967]:
type(test['reviews_date'][0][0])

str

In [1026]:
def t(row):
    new_row = []
    if len(row)>0:
        for i in row:
            new_row.append(pd.to_datetime(i,errors='coerce'))
        return new_row
    else:
        return []

In [1027]:
tt = test.copy()

In [1028]:
tt['reviews_date'] = tt['reviews_date'].apply(t)

In [997]:
type(tt['reviews_date'][0][0])

str

In [1029]:
tt['reviews_date']

0        [2017-12-31 00:00:00, 2017-11-20 00:00:00]
1        [2017-07-06 00:00:00, 2016-06-19 00:00:00]
2        [2018-01-08 00:00:00, 2018-01-06 00:00:00]
3                                                []
4        [2017-11-18 00:00:00, 2017-02-19 00:00:00]
                            ...                    
39995    [2017-12-16 00:00:00, 2017-11-12 00:00:00]
39996    [2017-12-21 00:00:00, 2017-12-12 00:00:00]
39997    [2016-11-03 00:00:00, 2008-04-12 00:00:00]
39998    [2017-07-11 00:00:00, 2017-06-18 00:00:00]
39999    [2016-01-31 00:00:00, 2012-07-04 00:00:00]
Name: reviews_date, Length: 40000, dtype: object

In [1044]:
for row in tt['reviews_date']:
    if len(row)==2:
        print((row[0]-row[1]).days)
    elif len(row) == 1:
        print(row)

41
382
2
272
108
-93
613
225
23
11
56
78
[Timestamp('2017-04-12 00:00:00')]
120
299
243
8
98
57
89
[Timestamp('2016-07-05 00:00:00')]
[Timestamp('2017-01-26 00:00:00')]
[Timestamp('2011-08-20 00:00:00')]
27
241
[Timestamp('2015-10-26 00:00:00')]
194
489
[Timestamp('2015-03-08 00:00:00')]
[Timestamp('2017-05-12 00:00:00')]
81
100
-299
164
1
-180
32
25
82
51
0
[Timestamp('2016-03-19 00:00:00')]
[Timestamp('2014-11-10 00:00:00')]
[Timestamp('2012-04-13 00:00:00')]
66
88
106
0
4
357
51
11
193
698
[Timestamp('2014-12-09 00:00:00')]
1
256
35
12
99
[Timestamp('2016-09-04 00:00:00')]
5
75
373
1
140
52
423
65
445
9
56
[Timestamp('2016-10-14 00:00:00')]
94
79
136
100
17
124
145
248
189
129
84
33
664
22
445
288
3
1086
112
120
80
21
18
98
308
0
41
40
48
8
204
25
[Timestamp('2014-08-04 00:00:00')]
43
147
169
739
3
139
128
-225
257
8
46
121
0
18
70
6
188
[Timestamp('2017-12-07 00:00:00')]
[Timestamp('2018-02-09 00:00:00')]
9
36
32
49
46
8
0
11
294
63
59
235
48
[Timestamp('2013-03-16 00:00:00')]
-214

132
[Timestamp('2016-11-03 00:00:00')]
91
285
0
1
86
-149
63
60
514
26
2
271
-72
8
1
[Timestamp('2016-07-09 00:00:00')]
[Timestamp('2017-04-11 00:00:00')]
145
1
91
144
-99
24
82
210
1355
46
40
17
30
72
58
538
7
114
144
173
62
[Timestamp('2017-07-14 00:00:00')]
6
25
30
[Timestamp('2016-08-28 00:00:00')]
114
196
145
[Timestamp('2015-06-24 00:00:00')]
12
375
40
110
207
773
3
[Timestamp('2016-12-15 00:00:00')]
12
[Timestamp('2017-10-21 00:00:00')]
[Timestamp('2017-01-05 00:00:00')]
48
101
430
4
33
205
594
62
-223
54
-116
680
8
99
7
4
83
21
11
39
558
32
19
1074
[Timestamp('2016-08-20 00:00:00')]
[Timestamp('2016-04-26 00:00:00')]
1
59
349
6
25
17
0
22
27
101
18
[Timestamp('2017-08-23 00:00:00')]
263
136
-55
25
183
31
26
141
2
225
101
1
26
38
37
16
529
10
36
183
28
61
279
217
166
35
92
46
14
[Timestamp('2015-09-17 00:00:00')]
42
339
[Timestamp('2012-03-21 00:00:00')]
[Timestamp('2016-07-17 00:00:00')]
37
113
4
91
[Timestamp('2016-09-05 00:00:00')]
0
7
63
436
[Timestamp('2014-06-20 00:00:00')

-282
7
17
379
153
190
[Timestamp('2018-01-06 00:00:00')]
104
[Timestamp('2017-08-06 00:00:00')]
[Timestamp('2017-07-16 00:00:00')]
43
-215
635
[Timestamp('2017-12-04 00:00:00')]
[Timestamp('2017-01-13 00:00:00')]
12
6
78
43
-285
166
[Timestamp('2017-01-21 00:00:00')]
290
115
49
33
-120
-241
128
7
128
45
468
87
101
3
63
77
134
200
1
64
105
26
96
3
30
1818
[Timestamp('2014-10-13 00:00:00')]
91
21
309
144
80
32
34
443
29
[Timestamp('2013-10-11 00:00:00')]
-200
[Timestamp('2016-01-15 00:00:00')]
143
[Timestamp('2013-09-03 00:00:00')]
26
55
26
95
[Timestamp('2017-04-10 00:00:00')]
-325
114
296
82
361
398
167
300
60
6
88
5
2
141
724
179
10
30
76
23
20
[Timestamp('2017-06-21 00:00:00')]
35
[Timestamp('2015-11-30 00:00:00')]
27
1
133
124
38
229
24
216
993
78
[Timestamp('2016-03-03 00:00:00')]
64
46
[Timestamp('2014-09-11 00:00:00')]
28
40
[Timestamp('2016-12-01 00:00:00')]
143
3
133
2
33
4
44
21
105
10
2
12
11
[Timestamp('2017-09-26 00:00:00')]
[Timestamp('2015-11-09 00:00:00')]
[Timestamp('20

169
17
[Timestamp('2017-08-28 00:00:00')]
[Timestamp('2015-12-24 00:00:00')]
72
-1
110
[Timestamp('2015-04-28 00:00:00')]
65
8
700
104
149
170
[Timestamp('2015-09-23 00:00:00')]
33
33
97
-259
1
[Timestamp('2016-10-03 00:00:00')]
108
3
688
[Timestamp('2016-07-03 00:00:00')]
135
10
241
39
164
708
85
293
81
14
108
172
26
195
188
[Timestamp('2017-11-24 00:00:00')]
83
-136
28
8
3
[Timestamp('2017-06-01 00:00:00')]
229
243
[Timestamp('2017-11-03 00:00:00')]
[Timestamp('2017-11-20 00:00:00')]
20
-115
3
1
8
162
99
547
[Timestamp('2015-09-25 00:00:00')]
28
2
128
687
789
[Timestamp('2017-11-21 00:00:00')]
269
44
49
[Timestamp('2017-03-06 00:00:00')]
69
155
51
539
271
777
0
33
153
[Timestamp('2017-07-16 00:00:00')]
0
113
134
-145
308
148
97
1490
527
1093
27
128
47
-31
921
27
174
[Timestamp('2016-09-24 00:00:00')]
630
88
37
167
22
279
-13
34
140
[Timestamp('2016-04-20 00:00:00')]
[Timestamp('2011-11-23 00:00:00')]
76
[Timestamp('2016-07-16 00:00:00')]
220
312
75
222
50
159
[Timestamp('2014-06-29 0

2
18
341
40
10
37
1
69
160
179
1
19
76
4
30
24
[Timestamp('2016-08-06 00:00:00')]
14
-200
43
85
2
[Timestamp('2017-02-12 00:00:00')]
60
16
90
[Timestamp('2014-06-13 00:00:00')]
108
[Timestamp('2016-05-11 00:00:00')]
[Timestamp('2015-07-07 00:00:00')]
16
52
151
135
34
[Timestamp('2017-12-02 00:00:00')]
232
103
0
162
37
209
103
109
163
62
121
-103
[Timestamp('2017-04-25 00:00:00')]
49
244
623
82
34
278
92
[Timestamp('2016-06-11 00:00:00')]
38
123
167
86
28
4
70
101
18
71
86
56
8
53
25
81
708
262
279
[Timestamp('2014-06-02 00:00:00')]
[Timestamp('2015-06-02 00:00:00')]
[Timestamp('2017-09-11 00:00:00')]
90
14
573
0
42
90
10
1
12
26
11
58
[Timestamp('2017-05-12 00:00:00')]
19
30
261
14
5
1
41
9
536
[Timestamp('2017-03-13 00:00:00')]
526
[Timestamp('2015-03-05 00:00:00')]
5
79
17
23
-60
25
20
257
23
5
12
153
66
461
68
193
111
26
25
[Timestamp('2017-05-31 00:00:00')]
604
80
105
[Timestamp('2017-07-20 00:00:00')]
8
14
[Timestamp('2016-08-18 00:00:00')]
489
86
36
1467
87
88
2
134
18
235
15
35


...

5
373
145
73
218
84
41
267
145
65
88
294
24
69
93
675
1097
71
-196
10
184
459
33
47
[Timestamp('2015-10-09 00:00:00')]
297
162
18
3
57
629
11
-59
93
1457
21
301
2
153
[Timestamp('2018-01-13 00:00:00')]
4
318
-33
53
[Timestamp('2016-03-21 00:00:00')]
[Timestamp('2017-04-11 00:00:00')]
244
35
[Timestamp('2015-05-09 00:00:00')]
-70
31
63
36
[Timestamp('2011-11-23 00:00:00')]
[Timestamp('2016-08-31 00:00:00')]
116
196
15
76
11
310
[Timestamp('2014-11-21 00:00:00')]
47
105
10
77
855
-95
29
[Timestamp('2017-09-07 00:00:00')]
81
121
573
12
26
9
0
33
23
1
280
15
311
28
53
[Timestamp('2016-01-21 00:00:00')]
974
45
48
6
-187
23
114
-226
14
28
[Timestamp('2017-11-13 00:00:00')]
1534
47
54
1
209
[Timestamp('2016-08-04 00:00:00')]
600
37
-150
13
236
0
48
18
15
109
105
2
10
[Timestamp('2017-12-16 00:00:00')]
33
55
7
[Timestamp('2014-11-10 00:00:00')]
[Timestamp('2017-03-17 00:00:00')]
11
9
41
[Timestamp('2015-07-16 00:00:00')]
122
39
168
239
31
1
305
138
65
[Timestamp('2017-10-22 00:00:00')]
[Timest

14
22
170
[Timestamp('2016-09-09 00:00:00')]
54
374
[Timestamp('2017-04-06 00:00:00')]
[Timestamp('2016-06-27 00:00:00')]
252
393
101
130
1
12
11
566
5
[Timestamp('2017-07-25 00:00:00')]
180
124
367
55
161
12
47
78
[Timestamp('2014-09-03 00:00:00')]
10
2940
12
6
446
23
539
127
484
3
13
22
91
-110
91
328
224
16
676
31
100
254
26
36
25
143
278
1
1741
57
292
[Timestamp('2012-09-25 00:00:00')]
162
77
46
44
376
359
[Timestamp('2017-08-02 00:00:00')]
199
103
-51
93
[Timestamp('2017-06-24 00:00:00')]
[Timestamp('2017-09-09 00:00:00')]
35
12
[Timestamp('2015-05-15 00:00:00')]
[Timestamp('2014-07-29 00:00:00')]
62
139
19
46
227
139
58
42
[Timestamp('2017-02-10 00:00:00')]
[Timestamp('2014-09-12 00:00:00')]
45
40
2
26
110
5
[Timestamp('2015-01-31 00:00:00')]
239
228
2
217
1
34
89
1
-268
434
86
7
130
[Timestamp('2017-02-21 00:00:00')]
[Timestamp('2013-05-14 00:00:00')]
5
24
-64
7
51
42
40
79
[Timestamp('2017-10-08 00:00:00')]
2
1
2
[Timestamp('2015-09-17 00:00:00')]
4
990
31
10
-99
[Timestamp('20

[Timestamp('2016-11-01 00:00:00')]
512
[Timestamp('2016-02-10 00:00:00')]
2
433
292
[Timestamp('2017-08-22 00:00:00')]
34
30
276
414
35
320
13
[Timestamp('2015-12-15 00:00:00')]
[Timestamp('2016-11-01 00:00:00')]
-189
57
8
17
4
76
70
[Timestamp('2016-06-12 00:00:00')]
7
35
81
25
711
-50
64
1
102
88
[Timestamp('2014-08-18 00:00:00')]
20
19
164
17
247
46
[Timestamp('2017-06-01 00:00:00')]
13
69
[Timestamp('2016-06-23 00:00:00')]
133
331
45
58
214
430
6
95
135
136
1
0
[Timestamp('2015-06-28 00:00:00')]
76
-3
[Timestamp('2016-12-03 00:00:00')]
339
24
-110
61
2
9
396
[Timestamp('2014-01-31 00:00:00')]
20
148
1
3
73
46
303
352
64
[Timestamp('2016-10-31 00:00:00')]
176
16
121
33
90
654
[Timestamp('2016-10-15 00:00:00')]
[Timestamp('2017-03-28 00:00:00')]
8
96
328
18
[Timestamp('2017-04-09 00:00:00')]
138
239
37
381
31
147
2
153
[Timestamp('2016-11-17 00:00:00')]
170
8
8
372
2
[Timestamp('2017-08-28 00:00:00')]
1195
478
-159
446
21
82
188
[Timestamp('2014-09-01 00:00:00')]
[Timestamp('2016-07-

[Timestamp('2017-04-30 00:00:00')]
24
16
8
157
1128
142
183
222
22
79
58
[Timestamp('2017-11-13 00:00:00')]
[Timestamp('2017-06-08 00:00:00')]
482
67
75
588
115
[Timestamp('2016-10-22 00:00:00')]
113
[Timestamp('2016-08-23 00:00:00')]
59
[Timestamp('2017-01-06 00:00:00')]
275
90
87
24
25
176
142
-247
4
97
18
340
[Timestamp('2017-02-15 00:00:00')]
29
[Timestamp('2016-04-15 00:00:00')]
38
[Timestamp('2014-12-31 00:00:00')]
385
[Timestamp('2017-10-21 00:00:00')]
-13
224
195
60
[Timestamp('2017-07-17 00:00:00')]
151
[Timestamp('2015-03-17 00:00:00')]
61
64
-239
23
123
32
47
79
311
33
7
29
8
8
9
1716
5
75
12
2
143
13
87
19
[Timestamp('2016-10-25 00:00:00')]
3
232
350
61
190
[Timestamp('2017-01-31 00:00:00')]
2
20
38
31
51
[Timestamp('2014-09-16 00:00:00')]
695
151
-206
61
8
40
14
17
649
88
1
12
15
6
240
12
233
68
5
41
160
28
[Timestamp('2016-10-18 00:00:00')]
255
166
45
193
8
65
[Timestamp('2017-09-06 00:00:00')]
27
46
19
1670
9
5
[Timestamp('2017-04-22 00:00:00')]
[Timestamp('2017-10-08 00

97
0
72
[Timestamp('2017-09-13 00:00:00')]
[Timestamp('2017-11-30 00:00:00')]
15
0
123
65
3
356
111
277
16
80
35
12
7
50
114
301
15
240
190
6
102
46
84
-177
93
36
11
49
25
140
2
108
3
[Timestamp('2017-03-19 00:00:00')]
18
1
5
22
[Timestamp('2015-01-26 00:00:00')]
8
642
35
15
[Timestamp('2017-07-28 00:00:00')]
10
59
17
22
88
[Timestamp('2015-06-01 00:00:00')]
78
29
3
478
176
26
618
78
38
10
0
333
196
[Timestamp('2017-11-11 00:00:00')]
47
122
[Timestamp('2017-10-08 00:00:00')]
249
170
715
300
56
[Timestamp('2016-06-11 00:00:00')]
8
2
-215
29
[Timestamp('2015-11-01 00:00:00')]
49
78
217
278
73
278
201
1048
[Timestamp('2017-09-21 00:00:00')]
582
60
9
36
10
60
396
231
26
1879
[Timestamp('2017-10-05 00:00:00')]
70
38
25
9
349
32
68
108
123
38
1
731
368
110
29
5
132
1
648
5
[Timestamp('2014-05-25 00:00:00')]
[Timestamp('2016-12-04 00:00:00')]
121
596
108
98
2
178
[Timestamp('2016-05-30 00:00:00')]
1
38
117
[Timestamp('2012-10-14 00:00:00')]
39
29
55
[Timestamp('2017-04-24 00:00:00')]
224
226


In [1045]:
def delta_datetime(row):
    if len(row) == 2:
        return (row[0]-row[1]).days
    else:
        return 0

In [1052]:
tt['delta_datetime'] = tt['reviews_date'].apply(delta_datetime)

In [1056]:
tt['delta_datetime'].sort_values(ascending=False)

7990     3207
10997    3137
39997    3127
8356     3096
19438    3094
         ... 
28483    -350
20064    -350
16050    -352
10823    -354
6288     -362
Name: delta_datetime, Length: 40000, dtype: int64

In [1057]:
tt

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta,reviews_date,last_reviews_date,delta_datetime
0,id_5569,Paris,"[European, French, International]",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,"[2017-12-31 00:00:00, 2017-11-20 00:00:00]",2017-12-31,41
1,id_1535,Stockholm,[Unknown],1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,"[2017-07-06 00:00:00, 2016-06-19 00:00:00]",2017-07-06,382
2,id_352,London,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,"[2018-01-08 00:00:00, 2018-01-06 00:00:00]",2018-01-08,2
3,id_3456,Berlin,[Unknown],3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,[],1970-01-01,0
4,id_615,Munich,"[German, Central European, Vegetarian Friendly]",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,"[2017-11-18 00:00:00, 2017-02-19 00:00:00]",2017-11-18,272
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"[Italian, Vegetarian Friendly, Vegan Options, ...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,"[2017-12-16 00:00:00, 2017-11-12 00:00:00]",2017-12-16,34
39996,id_6340,Paris,"[French, American, Bar, European, Vegetarian F...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,"[2017-12-21 00:00:00, 2017-12-12 00:00:00]",2017-12-21,9
39997,id_1649,Stockholm,"[Japanese, Sushi]",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,"[2016-11-03 00:00:00, 2008-04-12 00:00:00]",2016-11-03,3127
39998,id_640,Warsaw,"[Polish, European, Eastern European, Central E...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,"[2017-07-11 00:00:00, 2017-06-18 00:00:00]",2017-07-11,23


In [1058]:
tt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      40000 non-null  object 
 1   city               40000 non-null  object 
 2   cuisine_style      40000 non-null  object 
 3   ranking            40000 non-null  float64
 4   rating             40000 non-null  float64
 5   price_range        26114 non-null  object 
 6   number_of_reviews  37457 non-null  float64
 7   reviews            40000 non-null  object 
 8   url_ta             40000 non-null  object 
 9   id_ta              40000 non-null  object 
 10  reviews_date       40000 non-null  object 
 11  last_reviews_date  40000 non-null  object 
 12  delta_datetime     40000 non-null  int64  
dtypes: float64(3), int64(1), object(9)
memory usage: 4.0+ MB
